Carico tutte le librerie necessarie per stopwords, tokenizzazione e conteggi. La parte di position tag l'ho ripresa per l'inglese, ma non l'ho utilizzata nel nostro caso.

In [2]:
import string
import sys
import io 
import nltk

# Stopwords
from gensim.parsing.preprocessing import remove_stopwords
from nltk.corpus import stopwords 
nltk.download('stopwords')
stops = set(stopwords.words('italian'))
nltk_stopwords = nltk.corpus.stopwords.words('italian')
# Valutare di aggiungere eventuali forme arcariche delle stopwords italiane, o se c'è necessità di estendere questa lista
  
# Divido il testo in frasi in base ai punti
nltk.download('punkt')

# Tokenizer
from nltk.tokenize import word_tokenize 

# Per il discorso lemmatizzazione dobbiamo valutare come muoverci con l'italiano

# Lemmatization
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

# Position tag
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

# Contatore parole uniche
from collections import Counter

# Per esplorare risultati
import random

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\giorg\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\giorg\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\giorg\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\giorg\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\giorg\AppData\Roaming\nltk_data...


Valuto se la lista alternativa che ho trovato contiene più stopwords di quella di NLTK.

In [5]:
# Utilizzare questo se necessario aggiungere stopwords da altre liste

#other_stopwords = []
#with open('ita_stopword.txt') as f:
#    for line in f:
#        other_stopwords.append(line)
        
#other_stopwords = [s.replace('\n', '') for s in other_stopwords]

# Unisco la lista di stopwords di nltk con quella estratta dal .txt
#stopwords = list(nltk_stopwords)
#stopwords.extend(x for x in other_stopwords if x not in stopwords)

Definisco le varie funzioni di preprocessing: senza lemmatizzazione e con lemmatizzazione, usando, nell'ordine, NLTK, Simplemma e SpaCy.

In [6]:
# Preprocessing (NO lemmatization)
def preprocessing(file_name):

  output=""
  with open(file_name, encoding='utf-8') as f:
      for line in f:
          if not line.isspace(): # Rimuovo linee vuote
              output+=line

  # Divido il testo in frasi, basandomi sui punti
  output_sentences = nltk.tokenize.sent_tokenize(output)

  # Valutare se è necessario eliminare delle righe all'inizio o alla fine dei .txt se presentano licenze,...
  # output_sentences = output_sentences[:-]

  filtered_sentences = []
  # 'Pulisco' ogni frasi, una alla volta
  for sentence in output_sentences:
    # Metto tutto in lower case
    lower_sentence=sentence.lower()
    # Rimuovo caratteri non alfa numerici
    noalfa_sentence = [w for w in word_tokenize(lower_sentence) if (w.isalpha()==True)]
    # Rimuovo le stopwords e le parole di un solo carattere che potrebbero non essere incluse nella lista delle stopwords
    filtered_sentence = [w for w in noalfa_sentence if ((w not in stops) and (len(w) > 1))]
    # Ricostruisco la lista con le frasi 'pulite'
    if filtered_sentence:
      filtered_sentences.append(filtered_sentence)

  return filtered_sentences

In [ ]:
# Preprocessing (CON lemmatization)

def preprocessing_lem(file_name):

  output=""
  with open(file_name, encoding='utf-8') as f:
      for line in f:
          if not line.isspace(): # Rimuovo linee vuote
              output+=line

  # Divido il testo in frasi, basandomi sui punti
  output_sentences = nltk.tokenize.sent_tokenize(output)

  # Valutare se è necessario eliminare delle righe all'inizio o alla fine dei .txt se presentano licenze, ...
  # output_sentences = output_sentences[:-]

  filtered_sentences = []
  # 'Pulisco' ogni frasi, una alla volta
  for sentence in output_sentences:
    # Metto tutto in lower case
    lower_sentence=sentence.lower()
    # Rimuovo caratteri non alfa numerici
    noalfa_sentence = [w for w in word_tokenize(lower_sentence) if (w.isalpha()==True)]
    # Lemmatize
    lemmatized_sentence = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in noalfa_sentence]
    # Rimuovo le stopwords e le parole di un solo carattere che potrebbero non essere incluse nella lista delle stopwords
    filtered_sentence = [w for w in lemmatized_sentence if ((w not in stops) and (len(w) > 1))]
    # Ricostruisco la lista con le frasi 'pulite'
    if filtered_sentence:
      filtered_sentences.append(filtered_sentence)

  return filtered_sentences

In [ ]:
import simplemma

# Preprocessing (CON lemmatization w/ SIMPLEMMA)

def preprocessing_simplemma(file_name):

  output=""
  with open(file_name, encoding='utf-8') as f:
      for line in f:
          if not line.isspace(): # Rimuovo linee vuote
              output+=line

  # Divido il testo in frasi, basandomi sui punti
  output_sentences = nltk.tokenize.sent_tokenize(output)

  # Valutare se è necessario eliminare delle righe all'inizio o alla fine dei .txt se presentano licenze, ...
  # output_sentences = output_sentences[:-]

  filtered_sentences = []
  # 'Pulisco' ogni frasi, una alla volta
  for sentence in output_sentences:
    # Metto tutto in lower case
    lower_sentence=sentence.lower()
    # Rimuovo caratteri non alfa numerici
    noalfa_sentence = [w for w in word_tokenize(lower_sentence) if (w.isalpha()==True)]
    # Lemmatize
    lemmatized_sentence = [simplemma.lemmatize(w, lang='it') for w in noalfa_sentence]
    # Rimuovo le stopwords e le parole di un solo carattere che potrebbero non essere incluse nella lista delle stopwords
    filtered_sentence = [w for w in lemmatized_sentence if ((w not in stops) and (len(w) > 1))]
    # Ricostruisco la lista con le frasi 'pulite'
    if filtered_sentence:
      filtered_sentences.append(filtered_sentence)

  return filtered_sentences

In [ ]:
import spacy
import it_core_news_lg

nlp = it_core_news_lg.load()

# Preprocessing (CON lemmatization w/ SPACY). In questo caso ho rimosso le stopwords prima della lemmatizzazione, va provato
# con tutti a fare così

def preprocessing_spacy(file_name):

  output=""
  with open(file_name, encoding='utf-8') as f:
      for line in f:
          if not line.isspace(): # Rimuovo linee vuote
              output+=line

  # Divido il testo in frasi, basandomi sui punti
  output_sentences = nltk.tokenize.sent_tokenize(output)

  # Valutare se è necessario eliminare delle righe all'inizio o alla fine dei .txt se presentano licenze, ...
  # output_sentences = output_sentences[:-]

  filtered_sentences = []
  # 'Pulisco' ogni frasi, una alla volta
  for sentence in output_sentences:
    # Metto tutto in lower case
    lower_sentence=sentence.lower()
    # Rimuovo caratteri non alfa numerici
    noalfa_sentence = [w for w in word_tokenize(lower_sentence) if (w.isalpha()==True)]
    # Rimuovo le stopwords e le parole di un solo carattere che potrebbero non essere incluse nella lista delle stopwords
    nostop_sentence = [w for w in noalfa_sentence if ((w not in stops) and (len(w) > 1))]
    # Lemmatize
    lemmatized_sentence = " ".join([w for w in nostop_sentence])
    doc = nlp(lemmatized_sentence)
    filtered_sentence = [w.lemma_ for w in doc]
    # Ricostruisco la lista con le frasi 'pulite'
    if filtered_sentence:
      filtered_sentences.append(filtered_sentence)

  return filtered_sentences

Definisco la funzione per valutare i risultati del preprocessing.

In [ ]:
# Funzione per esplorare i risultati
def checkresults(sentences):
  words = []
  for i in range(len(sentences)):
    for j in range(len(sentences[i])):
      words.append(sentences[i][j])

  # Numero totale di parole
  print('Parole: ' + str(len(words)))

  # Numero di parole uniche
  word_count = Counter(words)
  keys = word_count.keys()
  print('Parole uniche: '+ str(len(keys)))

  # Parole più comuni
  print('Parole più comuni:')
  print(word_count.most_common(10))

  # Stampo alcune frasi random
  print('Frasi random:')
  randomlist = random.sample(range(0, len(sentences)), 5)
  for i in randomlist:
    print(sentences[i])

In [ ]:
def savesentences(filename, sentences):
  with open(filename + '.txt', 'w') as fp:
    for sentence in sentences:
      fp.write(str(sentence) + '\n')

In [ ]:
import os
os.chdir('Books')

In [ ]:
frasi_decameron = preprocessing('Decameron.txt')
frasi_decameron_lem = preprocessing_lem('Decameron.txt')
frasi_decameron_simp = preprocessing_simplemma('Decameron.txt')
frasi_decameron_spacy = preprocessing_spacy('Decameron.txt')

In [ ]:
checkresults(frasi_decameron)
checkresults(frasi_decameron_lem)
checkresults(frasi_decameron_simp)
checkresults(frasi_decameron_spacy)

In [ ]:
os.chdir('Sentences')

savesentences('frasi_decameron', frasi_decameron)
savesentences('frasi_decameron_lem', frasi_decameron_lem)
savesentences('frasi_decameron_simp', frasi_decameron_simp)
savesentences('frasi_decameron_spacy', frasi_decameron_spacy)

In [ ]:
print(frasi_decameron[0:1], frasi_decameron_lem[0:1], frasi_decameron_simp[0:1], frasi_decameron_spacy[0:1])

In [ ]:
# Mi ri-sposto in Books
os.chdir("..")

In [ ]:
frasi_divina = preprocessing('Divina Commedia.txt')
frasi_novelle = preprocessing('Novelle.txt')
frasi_orlando = preprocessing('Orlando furioso.txt')

In [ ]:
os.chdir('Sentences')

savesentences('frasi_divina', frasi_divina)
savesentences('frasi_novelle', frasi_novelle)
savesentences('frasi_orlando', frasi_orlando)

### Topic modelling

Si tratta di una cosa che potrebbe avere senso provare a fare mettendo insieme più libri, quindi costruendo i vari corpus in base all'autore, genere letterario o altro. Non ha senso farlo con le frasi di un singolo libro, in quanto, come si vede sotto, le parole più rilevanti risulterebbero quelle usate per costruire frasi di senso compiuto, come verbi o preposizioni. Invece, usando dei corpus che raccolgono più libri, sulla base delle statistiche della parole presenti in ciascun documento rispetto al totale, il modello riesce a individuare l'equilibrio fra i vari argomenti e valutare quali sono i più rilevanti.

In [ ]:
import gensim
from gensim.utils import simple_preprocess

In [ ]:
decameron_sentences = open('decameron_frasi.txt').read().splitlines()

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True rimuove la punteggiatura
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [ ]:
decameron_words = list(sent_to_words(decameron_sentences))

In [ ]:
print(decameron_words[:1][0][:30])

In [ ]:
import gensim.corpora as corpora

# Creo un dizionario
id2word = corpora.Dictionary(decameron_words)
# Creo un corpus
texts = decameron_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
from pprint import pprint

# Numero di topics
num_topics = 10
# Definisco il modello LDA
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Stampo le keyword associate ai topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
decameron_sentences[0]

# TENTATIVO W2V

In [ ]:
import os
from gensim.models import Word2Vec
from collections import Counter

In [ ]:
#os.chdir('Books/Sentences')#, togliere il commento se necessario
#os.chdir("..")

In [ ]:
def training_W2V(sentences, book):
  #addestro 2 modelli per libro, si può estendere tranquillamente il numero volendo

  for it in range(2):
    model = Word2Vec(sentences = sentences,
                    #window = 5, default value
                    min_count=10, #not consider word with absolute frequency <10 
                    vector_size=300, #vector size 
                    sg = 1, #skipgram algorithm
                    hs = 0,
                    negative = 5, #negative sampling with 5 noise words
                    workers = 5, #faster process
                    epochs = 6 #6 iterations
                    )
  
    model.save(book.lower() + "_" + str(it) + ".model")

In [ ]:
book_dict = {'Decameron.txt' : 'Decameron', 'Divina Commedia.txt' : 'Divina Commedia', 'Novelle.txt' : 'Novelle',
             'Orlando furioso.txt' : 'Orlando Furioso'}

for book in book_dict.keys():
    sentences = preprocessing(book)
    training_W2V(sentences, book_dict[book])

In [ ]:
divina0_model = Word2Vec.load("divina commedia_0.model")

In [ ]:
divina0_model.wv.most_similar("amore")

# TENTATIVO CADE

In [1]:
from cade.cade import CADE
from gensim.models.word2vec import Word2Vec
import pandas as pd

In [2]:
import os
os.chdir('Books')

In [7]:
sentences_deca = preprocessing('Decameron.txt')

In [8]:
sentences_novelle = preprocessing('Novelle.txt')

In [9]:
with open('Sentences/prova_deca_CADE.txt', mode='w') as f:
    for s in sentences_deca:
        for w in s:
            f.write(w+" ")
        f.write('\n')

In [10]:
with open('Sentences/prova_novelle_CADE.txt', mode='w') as f:
    for s in sentences_novelle:
        for w in s:
            f.write(w+" ")
        f.write('\n')

In [11]:
os.chdir('Sentences')

In [15]:
# Creo il compass (non mi funziona !cat su notebook)

with open('prova_compass.txt', 'w') as outFile:
    with open('prova_deca_CADE.txt', 'r') as deca, open('prova_novelle_CADE.txt', 'r') as novelle:
        outFile.write(deca.read())
        outFile.write(novelle.read())

In [32]:
aligner = CADE(size=300)

In [33]:
aligner.train_compass("prova_compass.txt") ### NON SO COME RISOLVERE QUESTO ERRORE, SU COLAB SEMBRA FUNZIONARE

Training the compass from scratch.


TypeError: __init__() got an unexpected keyword argument 'size'

In [ ]:
slice_one = aligner.train_slice("prova_deca_CADE.txt", save=True)
slice_two = aligner.train_slice("prova_novelle_CADE.txt", save=True)

In [ ]:
model_deca = Word2Vec.load('model/prova_deca_CADE.model')
model_novelle = Word2Vec.load('model/prova_novelle_CADE.model')

In [ ]:
model_deca.wv.most_similar("donna")

In [ ]:
model_novelle.wv.most_similar("donna")

In [ ]:
lauretta = model_deca.wv['lauretta']
model_novelle.wv.most_similar(positive=[lauretta])